In [1]:
# import libraries
import pandas as pd
import numpy as np
import math
import random
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix  
import matplotlib.pyplot as plt
from time import time
import seaborn as sns

# Import the datasets
liverCSV = pd.read_csv('E:\\Thesis\\Indian_Liver_Patient.csv')

# Handling the missing values
df = pd.DataFrame(liverCSV)
dataset = df.fillna(method='ffill', axis = 0)


# start undersampling process

target = 'Liver_patients'

# dataset counts 1 or 0 
print(dataset[target].value_counts())

# minority class length i.e 1
minority_class_len = len(dataset[dataset[target] == 0])
print(minority_class_len)

# display the indices of majority class i.e 0
majority_class_indices = dataset[dataset[target] == 1].index
print(majority_class_indices)

# randomly picking up the majority class indicies i.e. 0 
random_majority_indices = np.random.choice(majority_class_indices, minority_class_len, replace=False)
print(len(random_majority_indices))

# displaing the minority class indices i.e. 1
minority_class_indices = dataset[dataset[target] == 0].index
print(minority_class_indices)

# concatenate the minority indices and random majority incices
under_sample_indices = np.concatenate([minority_class_indices, random_majority_indices])

# locate the under_sample_indices to datasets
under_sample = dataset.loc[under_sample_indices]

# Bar Garph
sns.countplot(x=target, data=under_sample)

# displaing the values of 0 and 1
print(under_sample[target].value_counts())

# end undersampling process

tData = np.asarray(under_sample.drop('Liver_patients',1))
tTarget = np.asarray(under_sample['Liver_patients'])

# Normalize Data
means = np.mean(tData, axis=0)
stds = np.std(tData, axis=0)
tData = (tData - means)/stds

X_train, X_test, y_train, y_test = train_test_split(tData,tTarget,test_size=0.20, random_state = 42)

#define the no of iterations and no of particles
#K = 0.3
c1 = 2.05
c2 = 2.05
c = c1+ c2
khi = 0.729 # inertia weight(W)
n_iterations = 50
n_particles = 100


# creating the function of SVM
#particle = (gamma,C) & kernel type = rbf
def fitness_function(position):
    svclassifier = SVC(kernel='rbf', gamma = position[0], C = position[1] )  
    svclassifier.fit(X_train, y_train)
    y_train_pred = svclassifier.predict(X_train)
    y_test_pred = svclassifier.predict(X_test)   
    return (confusion_matrix(y_train,y_train_pred)[0][1] + confusion_matrix(y_train,y_train_pred)[1][0])/np.sum(confusion_matrix(y_train,y_train_pred)), (confusion_matrix(y_test,y_test_pred)[0][1] + confusion_matrix(y_test,y_test_pred)[1][0])/np.sum(confusion_matrix(y_test,y_test_pred))

def accuracy(position):
    svclassifier = SVC(kernel='rbf', gamma = position[0], C = position[1] )  
    svclassifier.fit(X_train, y_train)
    accuracy = svclassifier.score(X_test, y_test)
    return (accuracy * 100, "%")

def finalreport(position):
    svclassifier = SVC(kernel='rbf', gamma = position[0], C = position[1] )  
    svclassifier.fit(X_train, y_train)
    y_pred = svclassifier.predict(X_test)
    return (metrics.classification_report(y_test, y_pred))

#plotting (x axis = gamma, y axis = C)
def plot(position):
    x = []
    y = []
    for i in range(0,len(particle_position_vector)):
        x.append(particle_position_vector[i][0])
        y.append(particle_position_vector[i][1])
    colors = (0,0,0)
    plt.scatter(x, y, c = colors, alpha = 0.1)
    plt.xlabel('gamma')
    plt.ylabel('C')
    plt.axis([0,1,0,10])
    plt.gca().set_aspect('equal', adjustable='box')
    return plt.show()


# PSO algorithm to optimized the SVM parameters
particle_position_vector = np.array([np.array([random.random(), random.random()*10]) for _ in range(n_particles)])
pbest_position = particle_position_vector
pbest_fitness_value = np.array([float('inf') for _ in range(n_particles)])
gbest_fitness_value = np.array([float('inf'), float('inf')])
gbest_position = np.array([float('inf'), float('inf')])

velocity_vector = ([np.array([0, 0]) for _ in range(n_particles)])
iteration = 0

while iteration < n_iterations:
    start = time()
    plot(particle_position_vector)
    for i in range(n_particles):
        fitness_cadidate = fitness_function(particle_position_vector[i])
        print("error of particle-",i+1,"is (training, test)", fitness_cadidate, " At (gamma, c): ", particle_position_vector[i])
        
        if(pbest_fitness_value[i] > fitness_cadidate[1]):
            pbest_fitness_value[i] = fitness_cadidate[1]
            pbest_position[i] = particle_position_vector[i]

        if(gbest_fitness_value[1] > fitness_cadidate[1]):
            gbest_fitness_value = fitness_cadidate
            gbest_position = particle_position_vector[i]
        elif(gbest_fitness_value[1] == fitness_cadidate[1] and gbest_fitness_value[0] > fitness_cadidate[0]):
            gbest_fitness_value = fitness_cadidate
            gbest_position = particle_position_vector[i]
            
    for i in range(n_particles):
        max_velocity = np.array([1,10])
        min_velocity = np.array([-1,-10])
        max_position = np.array([1,10])
        min_position = np.array([0.0001, 0.001])
        new_velocity = khi*(velocity_vector[i] + (c1*random.random()) * (pbest_position[i] - particle_position_vector[i]) + (c2*random.random()) * (gbest_position-particle_position_vector[i]))
        for j in range(0,len(new_velocity)):
            if(new_velocity[j] > max_velocity[j]):
                new_velocity[j] = max_velocity[j]
            elif(new_velocity[j] < min_velocity[j]):
                new_velocity[j] = min_velocity[j]
            
        new_position = new_velocity + particle_position_vector[i]
        particle_position_vector[i] = new_position
        for j in range(0,2):
            if(particle_position_vector[i][j] > max_position[j]):
                particle_position_vector[i][j] = max_position[j]
            elif(particle_position_vector[i][j] < min_position[j]):
                particle_position_vector[i][j] = min_position[j]
    
    iteration = iteration + 1
    stop = time()
    sec = stop-start
    print(sec)
print("The best position is ", gbest_position, "in iteration number", iteration, "with error rate (train, test):", fitness_function(gbest_position))
print("The accuracy of PSO-SVM:", accuracy(gbest_position))
print(finalreport(gbest_position))



'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.


1    416
0    167
Name: Liver_patients, dtype: int64
167
Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   9,  10,
            ...
            571, 572, 573, 574, 575, 576, 577, 579, 580, 581],
           dtype='int64', length=416)
167
Int64Index([  8,  12,  15,  17,  24,  28,  29,  32,  33,  34,
            ...
            539, 540, 541, 542, 545, 551, 564, 566, 578, 582],
           dtype='int64', length=167)
1    167
0    167
Name: Liver_patients, dtype: int64


<Figure size 640x480 with 1 Axes>

error of particle- 1 is (training, test) (0.21348314606741572, 0.2537313432835821)  At (gamma, c):  [0.28761011 0.81767352]
error of particle- 2 is (training, test) (0.0449438202247191, 0.40298507462686567)  At (gamma, c):  [0.77153682 5.65485168]
error of particle- 3 is (training, test) (0.15730337078651685, 0.23880597014925373)  At (gamma, c):  [0.52670567 1.57657104]
error of particle- 4 is (training, test) (0.03745318352059925, 0.417910447761194)  At (gamma, c):  [0.68521674 8.72346758]
error of particle- 5 is (training, test) (0.15730337078651685, 0.2537313432835821)  At (gamma, c):  [0.35934084 3.66866102]
error of particle- 6 is (training, test) (0.13108614232209737, 0.2835820895522388)  At (gamma, c):  [0.29960022 7.76956137]
error of particle- 7 is (training, test) (0.08614232209737828, 0.40298507462686567)  At (gamma, c):  [0.63008493 4.20990496]
error of particle- 8 is (training, test) (0.0749063670411985, 0.3880597014925373)  At (gamma, c):  [0.41903414 8.44220743]
error of

error of particle- 80 is (training, test) (0.1647940074906367, 0.26865671641791045)  At (gamma, c):  [0.67627936 0.65306868]
error of particle- 81 is (training, test) (0.20599250936329588, 0.3582089552238806)  At (gamma, c):  [0.65931301 0.40503431]
error of particle- 82 is (training, test) (0.08614232209737828, 0.40298507462686567)  At (gamma, c):  [0.75513163 3.12368868]
error of particle- 83 is (training, test) (0.25842696629213485, 0.23880597014925373)  At (gamma, c):  [0.02537845 8.53746245]
error of particle- 84 is (training, test) (0.1760299625468165, 0.26865671641791045)  At (gamma, c):  [0.65012493 0.7299066 ]
error of particle- 85 is (training, test) (0.033707865168539325, 0.40298507462686567)  At (gamma, c):  [0.76781852 9.35771998]
error of particle- 86 is (training, test) (0.07116104868913857, 0.40298507462686567)  At (gamma, c):  [0.50610412 6.28813651]
error of particle- 87 is (training, test) (0.03745318352059925, 0.3880597014925373)  At (gamma, c):  [0.90810694 4.17398

...

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.


error of particle- 92 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22309996]
error of particle- 93 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22309996]
error of particle- 94 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22309996]
error of particle- 95 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22309996]
error of particle- 96 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22309996]
error of particle- 97 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22309996]
error of particle- 98 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22309996]
error of particle- 99 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22

<Figure size 640x480 with 1 Axes>

error of particle- 1 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22309996]
error of particle- 2 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22309996]
error of particle- 3 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22309996]
error of particle- 4 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22309996]
error of particle- 5 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22309996]
error of particle- 6 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22309996]
error of particle- 7 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22309996]
error of particle- 8 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22309996]


error of particle- 67 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22309996]
error of particle- 68 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22309996]
error of particle- 69 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22309996]
error of particle- 70 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22309996]
error of particle- 71 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22309996]
error of particle- 72 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22309996]
error of particle- 73 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22309996]
error of particle- 74 is (training, test) (0.15355805243445692, 0.22388059701492538)  At (gamma, c):  [0.25139709 5.22